# Web Scrapping to CoE

## Import Libraries

In [ ]:
import sys
sys.path.insert(1, '../shared')
import requests
from bs4 import BeautifulSoup
from time import sleep

import pandas as pd

import numpy as np

import os.path
from os import path

from utils import verbosity

from typing_extensions import List, Dict

import pandas as pd
df = pd.DataFrame()



## Define WebScrapping atributes

In [ ]:
verbosity(msg=f'getting_link',
            tl=1)

auction_link = 'https://allianceforcoffeeexcellence.org/competition-auction-results/'

is_verbosity = True

## Define other classess

In [3]:
def find_number_in_string_array(array):
    for value in array:
        if value.isdigit():
            return value

class CharProcessing:
    
    @staticmethod
    def get_country_name(auction_link):
        auction_link=auction_link[:(len(auction_link)-1)]
        array_auction_split = auction_link.split('/')
        return array_auction_split[(len(array_auction_split))-1]
        
class SearchOperations:
    
    @staticmethod
    def get_country_by_name(array_countries, name):
        '''Returned argument is the country and link auction'''
        for country in array_countries:
            if(country['name']==name):
                return country


class SaveOperations:
    def __init__(self):
        pass
    
    
    def file_exist(self, file_name):
        
        return path.exists(f'../static/{file_name}')
    
    def save_html_page(self, html_code, file_name):
        verbosity(msg=f'enter in save_html_page function',
                    tl=1)
        verbosity(msg=f'checking if the file exist',
                    tl=2)
        if(self.file_exist(file_name)):
            verbosity(msg=f'the file exist',
                        tl=2)
            verbosity(msg=f'saving html page',
                        tl= 3)
            f = open(f'../static/{file_name}','w')
            f.write(html_code)
            f.close()
        else:
            verbosity(msg=f"the file doesn't exist",
                        tl=2)
            verbosity(msg=f"creating file",
                        tl=3)
            f = open(f'../static/{file_name}', "x")
            verbosity(msg=f"saving html page",
                        tl=3)
            f.write(html_code)
            f.close()
        verbosity(msg=f'out from save_html_page function',
                    tl=1)

class Page:
    
    
    def __init__(self, root_link:str):
        verbosity(msg=f'Page class was initialized with root_link: {root_link}',
                    tl=1)
        self.root_link = root_link
        

        
    def get_web_content(self, tl=2):
        ''' returned the web content of a link with Beautiful Soup '''
        verbosity(msg=f'getting web content', 
                    verb=is_verbosity, tl=tl)
        sleep(0.5)
        verbosity(msg=f'request page from link',
                    verb=is_verbosity, tl= tl+1)
        try:
            response = requests.get(self.root_link)
            verbosity(msg=f'Success request',
                        verb=is_verbosity, tl= tl+2)
        except Exception as e:
            verbosity(msg = f'error: {e}',
                    type='error',	verb=is_verbosity, tl= tl+1)
            
        verbosity(msg=f'returning content',
                    verb=is_verbosity, tl= tl+2)
        return BeautifulSoup(response.content, 'html.parser')
    
    
    def get_content_by_html_tags(self, beuty_object, html_tag):
        ''' get the web content of beautiful object by html tag '''
        return beuty_object.find_all(html_tag)
    
    
    def get_content_by_class(self,  beuty_object, class_name,):
        ''' get the web content of beautiful object by class '''
        return beuty_object.find_all(class_=class_name)
    
       
    def get_content_by_html_attr(self,  beuty_object, attribute_name, attribute_value):
        ''' get the web content of beautiful object by attribute '''
        return beuty_object.find_all(attrs={attribute_name : attribute_value})

## Define WebScrapping classess

### Class to Home Page

In [4]:
class MainPage (Page):
    
        
    def get_auction_links(self, web_content_main, id_main_menu):
        '''
        Summary: 
            get the links of auctions in main page
        
        Parameters:
            web_content_main: content of main page
            id_main_menu: id to search all links in main menu
            
        Returns:
            list with the auctions links
        
        '''
        links = web_content_main.find_all(id=id_main_menu)[0].find_all('a')
        array_auctions_links = []
        
        index = 0
        for link in links[:(len(links)-1)]:
            link_href = str(link.get('href'))
            if(link_href!='#' and link_href != 'None'):
                array_auctions_links.append(
                    {
                        'name': CharProcessing.get_country_name(link_href),
                        'link': link_href
                    }
                )
                                
                
                # array_auctions_links['name'].append(CharProcessing.get_country_name(link_href))
                # array_auctions_links['link'].append(link_href)
        return array_auctions_links
                
                # index += 1
            
        



### Attributes dictionary

In [5]:
class AttributesDictionary:
    
    
    def __init__(self):
        self.dict_auction_translator = {
            'Country': ['COUNTRY'],
            'Rank': ['RANK', 'Ranking', 'LOT NUMBER', 'Rank'],
            'Score': ['SCORE', 'Score'],
            'Farm':['FARM', 'Farm', 'Farm / CWS'],
            'Url_farm':[''],
            'Farmer':['FARMER', 'Farmer / Representative'],
            'Region':['REGION', 'Region'],
            'Weigth_(Kg)':['WEIGHT (KG)'],
            'Weigth_(Lb)':['Weight – lbs', 'Pounds'],
            'Variety':['VARIETY'],
            'Process':['PROCESS'],
            'High_Bid':['High Bid'],
            'Total_Value':['Total Value'],
            'Company_Name':['Company Name'],
            'Total_Comission':['TOTAL COMISSION'],
            'Size_30Kg_boxes':['Size'],
            'Altitude':[''],
            'History':[''],
            'Enviromental_Care':[''],
            'Coffee_Processing_Information':[''],
            'Annual_Production':[''],
            'Annual_Precipitation':[''],
            'Shade_Grown_Type':[''],
            'Water_Source':[''],
            'Variety':[''],
            'Coffee_Growing_Area':[''],
            'High_Bidders':[''],
            

        }



    def translate_attribute(self, value_attribute):
        name_attribute = ''
        for item in self.dict_auction_translator.items():
            for attribute in item[1]:
                if attribute == value_attribute:
                    name_attribute = item[0]
                    return name_attribute
                    

### Auction Data Class

In [6]:

class Auction:
    def __init__(self):
        self.dict_auction = {
            'Year': '',
            'Country': '',
            'Rank': '',
            'Score': '',
            'Farm':'',
            'Url_farm':'',
            'Farmer':'',
            'Region':'',
            'Weigth_(Kg)':'',
            'Weigth_(Lb)':'',
            'Variety':'',
            'Process':'',
            'High_Bid':'',
            'Total_Value':'',
            'Company_Name':'',
            'Total_Comission':'',
            'Size_30Kg_boxes':'',
            'Altitude':'',
            'History':'',
            'Enviromental_Care':'',
            'Coffee_Processing_Information':'',
            'Annual_Production':'',
            'Annual_Precipitation':'',
            'Shade_Grown_Type':'',
            'Water_Source':'',
            'Variety':'',
            'Coffee_Growing_Area':'',
            'High_Bidders':'',

        }

        
    def get_dict_auction(self):
        return self.dict_auction
    
    def set_dict_auction(self, dict_auction):
        self.dict_auction = dict_auction
    
        
    
    def __str__(self):
        string = ("Country: " +self.dict_auction['Country'] + 
                    "\n" + "Rank: " +self.dict_auction['Rank'] + 
                    "\n" + "Score: " + self.dict_auction['Score'] + 
                    "\n" + "Farm: " + self.dict_auction['Farm'] + 
                    "\n" + "Url_farm: " + self.dict_auction['Url_farm'] + 
                    "\n" + "Farmer: " + self.dict_auction['Farmer'] + 
                    "\n" + "Region: " + self.dict_auction['Region'] + 
                    "\n" + "Weigth_(Kg): " + self.dict_auction['Weigth_(Kg)'] + 
                    "\n" + "Weigth_(Lb): " + self.dict_auction['Weigth_(Lb)'] + 
                    "\n" + "Variety: " + self.dict_auction['Variety'] + 
                    "\n" + "Process: " + self.dict_auction['Process'] + 
                    "\n" + "High_Bid: " + self.dict_auction['High_Bid'] + 
                    "\n" + "Total_Value: " + self.dict_auction['Total_Value'] + 
                    "\n" + "Company_Name: " + self.dict_auction['Company_Name'] + 
                    "\n" + "Total_Comission: " + self.dict_auction['Total_Comission'] + 
                    "\n")
        return string
                
        
class AuctionData:
    
    def __init__(self):
        self.array_auctions = []
        self.auctions_number = 0
        self.tl = 3
    
    
    def set_auctions_number(self, auctions_number):
        self.auctions_number = auctions_number
    
    
    def get_auctions_number(self):
        return self.auctions_number 
    
    def compare_auction_index(self, auction, rank):
        if auction.get_dict_auction()['Rank'].lower() == rank.lower():
            return True
        else:
            return False

    
    def get_dict_auction_index_by_rank(self, rank):
        try:
            dict_auction_index = [self.compare_auction_index(auction, rank) for auction in self.array_auctions].index(True)
            return dict_auction_index
        
        except ValueError as e:
            # verbosity(f'Value Error in get_dict_auction_index_by_rank function: {e} in rank {rank} ', type='error', tl=self.tl)
            return 'False'
        
        except AttributeError as e:
            verbosity(f'Attribute Error in get_dict_auction_index_by_rank function: {e}', type='error', tl=3)
            return 'False'
    
    
    
    def refactor_array_auctions(self):
        # print(self.array_auctions)
        array_auctions_complete = []
        try:
            array_auctions_complete = [auction.get_dict_auction() for auction in self.array_auctions]
        except AttributeError:
            verbosity(msg=f'error detected in Auction object', type='Error', tl=1)
        
        return(array_auctions_complete)
        # print(self.array_auctions_complete)
    
    def get_data_titles(self):
        dict_auction = self.array_auctions[0].get_dict_auction()
        verbosity(msg=f'getting title_auctions',
                    tl=1)
        array_titles = [title_auction for title_auction in dict_auction.keys()]
        verbosity(msg=f'getting title_auctions - Success!',
                    tl=1)
        
        return array_titles

        

### Class to Auction Results Page

In [7]:

class AuctionResultsPage(Page):
    
    def __init__(self, dict_page_country:Dict):
        
        
        self.root_link = dict_page_country['link']
        
        array_name_page = dict_page_country['name'].split('-')
        self.country = array_name_page[0]
        self.year = find_number_in_string_array(array_name_page)
        verbosity(msg=f'Page class was initialized with. \n root_link: {self.root_link} \n country: {self.country} \n year: {self.year}',
                    tl=1)
    
    def get_table_titles (self, array_td, attributes_dictionary):
        array_titles = []
        for td in array_td:
            array_titles.append(attributes_dictionary.translate_attribute(td.text))
        return array_titles
    
    def get_auction_data(self, web_content_main, tl=3):
        verbosity(msg=f'getting auction data',
            verb=is_verbosity, tl=tl)
        auction_data = AuctionData()
        attributes_dictionary = AttributesDictionary()
        
        array_table_data = self.get_content_by_html_tags(web_content_main, 'tbody')
        array_title_table_data = self.get_content_by_html_tags(web_content_main, 'thead')
        index = 0
        for table_data in array_table_data:
            array_tr = self.get_content_by_html_tags(table_data, 'tr')

            is_initial_tr = True
            array_titles = []
            

            for tr in array_tr:
                array_td = self.get_content_by_html_tags(tr, 'td')
                is_fake = False
                auction_index = 'False'
                
                if is_initial_tr and len(array_title_table_data)!=0:
                    try:
                        array_th = self.get_content_by_html_tags(array_title_table_data[index], 'th')
                        index = index + 1

                        array_titles = self.get_table_titles(array_th, attributes_dictionary)
                        verbosity(msg=f'get tittles: {array_titles}',
                                    verb=is_verbosity, tl=tl+1)
                        is_initial_tr = False
                    except IndexError as e:
                        verbosity(msg=f"Found ", type='Error', tl=5)
                
                if is_initial_tr:
                    array_titles = self.get_table_titles(array_td, attributes_dictionary)
                    is_initial_tr = False
                    
                else:
                    auction = Auction()
                    for td, table_title in zip(array_td, array_titles):
                        if table_title == 'Farm':
                            a_tag = td.find('a', href=True) #self.get_content_by_html_tags(td, 'a')[0]
                            if(a_tag != None):
                                auction.dict_auction['Url_farm'] = a_tag['href']
                            auction.dict_auction[table_title] = td.text
                        else:
                            if table_title == 'Rank':
                                auction_index = auction_data.get_dict_auction_index_by_rank(td.text)    
                                
                                if (str(td.text)==''):
                                    is_fake = True
                                auction.dict_auction[table_title] = (td.text.lower())
                            
                            else:
                                auction.dict_auction[table_title] = td.text
                            
                            
                    if(not(is_fake)):
                        if(auction_index!='False'):
                            for key, value in auction.get_dict_auction().items():
                                if(value!=''):
                                    auction_data.array_auctions[auction_index].get_dict_auction()[key] = value
                                                                
                            # auction.set_dict_auction(auction_data.array_auctions[auction_index].get_dict_auction() | auction.get_dict_auction())
                        else:
                            auction.dict_auction['Country'] = self.country
                            auction.dict_auction['Year'] = self.year
                            try:
                                auction.get_dict_auction()['Rank'][0].isnumeric()
                                auction_data.array_auctions.append(auction)
                            except:
                                pass
                        
        return auction_data                      
            
            

## Main Program

In [ ]:
auction = Auction()
mainPage = MainPage(auction_link)
web_content_main = mainPage.get_web_content()   
array_auctions_links = mainPage.get_auction_links(web_content_main, 'menu-coe-country-programs-menu')
df_auctions = pd.DataFrame(columns=auction.dict_auction.keys())
array_auctions_links = [auction for auction in array_auctions_links if 'colombia' in auction['name'] ]
for country in array_auctions_links:
    auction_page = AuctionResultsPage(country)
    web_content_auction_page = auction_page.get_web_content()
    auction_data = auction_page.get_auction_data(web_content_auction_page)
    array_dict_auctions = auction_data.refactor_array_auctions()
    df = pd.DataFrame(array_dict_auctions)
    df_auctions = pd.concat([df_auctions, df], ignore_index=True)
    # verbosity(msg=f"df_auctions = \n {df_auctions.tail()}", tl=1)

df_auctions.to_excel(f'../auctions_download/Auctions.xlsx')


# mainPage = MainPage(auction_link)
# web_content_main = mainPage.get_web_content()   
# array_auctions_links = mainPage.get_auction_links(web_content_main, 'menu-coe-country-programs-menu')

In [9]:
df_auctions.to_excel(f'../auctions_download/Auctions.xlsx')


In [9]:
# print(array_auctions_links)

In [10]:
# auction = 'colombia-2022'
# country = SearchOperations.get_country_by_name(array_auctions_links, auction)

# auction_page = AuctionResultsPage(country)

# web_content_auction_page = auction_page.get_web_content()



In [11]:
# %%time
# auction_data = auction_page.get_auction_data(web_content_auction_page)
# # print(auction_data)
# # array_auctions[66].dict_auction

# array_dict_colombian_auctions = auction_data.refactor_array_auctions()

# print(array_dict_colombian_auctions)


# print([(dict_auction['Rank'].lower()) for dict_auction in array_dict_colombian_auctions])




In [12]:
# df_colombian_auctions = pd.DataFrame(array_dict_colombian_auctions)
# df_colombian_auctions.to_excel(f'../auctions_download/Auctions-{auction}.xlsx')

# print(auction_page.get_auction_data(web_content_auction_page))


In [13]:
# save_operations = SaveOperations()
# save_operations.save_html_page(html_code=str(web_content_auction_page), file_name='html_code_auction_page.html')


In [14]:
# table_auction = auction_page.get_content_by_html_attr(web_content_auction_page, 'data-mtr-content', 'WEIGHT (KG)')

In [15]:

# table_auction = auction_page.get_content_by_html_tags(web_content_auction_page, 'tr')

# row_auction = table_auction[1].find('td')
# print(row_auction.text)

# print(type(table_auction[0]))

# for i in table_auction:
#     print((str(i)).split("\n"))
